## Multi-Agent System Using Langchain and Gemini

### Install the necessary libraries using pip

>pip install langchain==0.3.24 langchain-google-genai==2.1.3 tavily-python streamlit

In [ ]:
import os
import csv

from api_key import api_key, tavily
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import Tool, initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory
from langchain.tools.tavily_search import TavilySearchResults

### Setup your environment with the necessary API Keys

Google Gemini API at: https://aistudio.google.com/

Tavily API Key at: https://app.tavily.com/home

1. Once you have your API keys open api-key.py
2. Fill the keys appropriately 

In [ ]:
os.environ["GOOGLE_API_KEY"] = api_key
os.environ["TAVILY_API_KEY"] = tavily

### Configure our LLM
Langchain allows you intergrate with different types of LLM and AI platforms like OpenAI, Anthropic, Hugging Face etc

For this lab we will be using Google Gemini

we setup the parameters
1. temperature which controls the creativity of our LLM ranging from 0.0 to 1.0
2. model which is the version of Gemini we intend to use
3. max_tokens, this is the response length limit of the model
4. top_p, controls the range and diversity of the possible next words the model considers

In [ ]:
llm = ChatGoogleGenerativeAI(
    temperature=0.7,
    model="gemini-2.0-flash-001",
    max_tokens=500,
    top_p=0.9,
)

#### Searching through our data file 
To find if the requested item and quantity are available, this will be used to create a custom tool for our inventory agent

In [ ]:
def check_inventory(item, quantity, csv_file="shop.csv"):
    """
    Checks the inventory for the availability of a specific item and quantity.

    Args:
      item (str): The name of the item to check in the inventory.
      quantity (int): The quantity of the item to check for availability.
      csv_file (str, optional): The path to the CSV file containing inventory data. 
                    Defaults to "shop.csv".

    Returns:
      str: A message indicating whether the requested item and quantity are available, 
         the available stock if insufficient, or an error message if an exception occurs.

    Notes:
      - The CSV file is expected to have columns "item" and "quantity".
      - The function performs a case-insensitive comparison for the item name.
    """
    try:
        with open(csv_file, newline='') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                if row["item"].lower() == item.lower():
                    available = int(row["quantity"])
                    if available >= int(quantity):
                        return f"{item.title()} is available. Requested: {quantity}, In stock: {available}."
                    else:
                        return f"Only {available} units of {item} are available. Requested: {quantity}."
        #return f"{item.title()} is not in stock."
    except Exception as e:
        return f"Error checking inventory: {str(e)}"

>Recieve text input and extracts the quantity and item from user input

In [ ]:
def inventory_check_natural_language(input_text):
    """
    Parses a natural language input string to extract an item and its quantity, 
    and checks the inventory for the specified item and quantity.

    The function looks for patterns in the input text that specify a quantity 
    followed by an item name, optionally followed by "are" or "is" and "available". 
    If a match is found, it extracts the quantity and item name, and calls the 
    `check_inventory` function with these values. If no match is found, it 
    returns an error message.

    Args:
      input_text (str): A natural language string describing the item and quantity 
                to check in the inventory.

    Returns:
      str: The result of the `check_inventory` function if parsing is successful, 
         or an error message if parsing fails.
    """
    import re
    match = re.search(r"(\d+)\s+(.+?)\s*(?:are|is)?\s*available", input_text.lower())
    if match:
        quantity = int(match.group(1))
        item = match.group(2).strip()
        return check_inventory(item, quantity)
    else:
        return "Could not parse item and quantity from input."

#### Create our inventory tool
1. we name our inventory tool "Inventory Checker
2. we pass our custom function as the action of this tool
3. the description of the tool is now clearly stated, this improves the effciency of the model

In [ ]:
inventory_tool = Tool(
    name="Inventory Checker",
    func=inventory_check_natural_language,
    description="Use this tool to check the inventory for a specific item and quantity. "
                "Input should be in the format: 'X items are available' or 'X items is available'. "
                "Example: '5 apples are available'."
)

#### Setup our Search tool
1. First create a search_tool property of TavilySearchResults
2. We set our max results to 3 (could be higher or less)
3. Then proceed to create the nutrition_tool of type Tool
4. Note that we set the function of this tool to **search_tool.run()** to start the tool

In [ ]:
search_tool = TavilySearchResults(max_results=3)
nutrition_tool = Tool(
    name="Nutrition Info",
    func=lambda query: search_tool.run(f"Nutritional information of {query}"),
    description="Gives the nutrition facts of a given item."
)

## Pheeew that has been a lot to take it
> Drink some water

## Time to create and equip our agents 

For the most part our models have similar features like:
>1. agent being set to CHAT_ZERO_SHOT_REACT_DESCRIPTION (An agent that interacts in a conversational manner and can use tools it has not been explicitly trained on by relying on the tools description) **there are lots of other types of Agent**
>2. memory set to ConversationBufferMemory(memory_key="chat_history") this way our model maintains the context of the current query with the key "chat_history"
>3. verbose set to False (we will set this to true later to view our model's reasoning)

Our conversation agent has no tools but our **inventory_agent has the inventory_tool** and the **nutrition_agent has the nutrition_tool**

In [ ]:
conversation_agent = initialize_agent(
    tools=[],
    llm=llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    memory=ConversationBufferMemory(memory_key="chat_history"),
    verbose=False,
)

In [ ]:
inventory_agent = initialize_agent(
    tools=[inventory_tool],
    llm=llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    memory=ConversationBufferMemory(memory_key="chat_history"),
    verbose=False,
)

In [ ]:
nutrition_agent = initialize_agent(
    tools=[nutrition_tool],
    llm=llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    memory=ConversationBufferMemory(memory_key="chat_history"),
    verbose=False,
)

## Running our Agents 🤖🤖🤖

In [ ]:
def run_agents(item: str, quantity: int):
    convo = conversation_agent.run(f"Tell me what an {item} is.")
    stock_query = f"Check if {quantity} {item}(s) are available in stock."
    stock = inventory_agent.run(stock_query)
    nutrition = nutrition_agent.run(item)
    return convo, stock, nutrition

# Congratulations!!! 🥳

### run your code with
> **streamlit run front_end.py**